In [1]:
from IPython.display import display, HTML

In [2]:
from gensim.models import Word2Vec

C:\Users\vasel\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
MODEL = 'Linguistics/LinguisticsModel'

In [4]:
wv_model = Word2Vec.load(MODEL)

In [5]:
color_scheme = {'genitives': '<b><font color="#A52A2A">{}</font></b>'
                , 'comparativ':'<span style="background-color: #66CDAA">{}</span>'
                , 'coordinate_NPs': '<span style="background-color: #228B22">{}</span>'
                , 'not in vocabulary': '<span style="background-color: #FFE4E1">{}</span>'
                , 'i vs we': '<b><font color="#8B008B">{}</font></b>'
                , 'imperative mood': '<span style="background-color: #DEB887">{}</span>'
                , 'subjunctive mood': '<span style="background-color: #BC8F8F">{}</span>'}

In [6]:
class HTMLStyle:
    def __init__(self, color_scheme=color_scheme):
        self.color_scheme = color_scheme

In [7]:
style = HTMLStyle()

In [8]:
strs = []
for k, v in color_scheme.items():
    strs.append(style.color_scheme[k].format(k))

### Цвет разных ошибок для примера

In [9]:
HTML('   '.join(strs))

In [21]:
from collections import defaultdict
from gensim.models import Word2Vec
from tqdm import tqdm
import logging
from alphabet_detector import AlphabetDetector
import json

MORE_LESS = ['более', 'менее']
STATISTICS = "maxs"
MODEL = 'Linguistics/LinguisticsModel'

class Searcher:
    def __init__(self):
        self.found = defaultdict(list)
        self.flag_i_vs_we = ''
        self.found_word = defaultdict(list)

    def find_genitives(self, gen_chain, word, s, i, threshold=6):
        if word['feats'] and 'Case' in word['feats'].keys() and word['feats']['Case'] == 'Gen':
            gen_chain.append((word['form'], s, i))
        else:
            if len(gen_chain) >= int(threshold):
                self.found['genitives'].append(gen_chain)
                for gen in gen_chain:
                    self.found_word[gen].append('genitives')
            gen_chain = []
        return gen_chain

    def find_wrong_comparativ(self, sent, word, i, s):
        if i+1 < len(sent):
            next = sent[i + 1]
            if word['form'] in MORE_LESS and 'comp' in next['feats']:
                self.found['comparatives'].append((word['form'], next['form'], s, i))
                self.found_word[(word['form'], s, i)].append('comparatives')

    #!
    def find_wrong_coordinate_NPs(self, sent, i, s, word, model):
        if i+1 < len(sent):
            if word['form'] == 'и':
                t = i
                pair = []
                while (sent[t]['feats'] and 'S' not in sent[t]['feats']) and (sent[t]['feats'] and 'V' not in sent[t]['feats']) and t > 0:
                    t -= 1
                if sent[t]['feats'] and 'S' in sent[t]['feats']:
                    pair.append(sent[t]['form'])
                t = i
                while (sent[t]['feats'] and 'S' not in sent[t]['feats']) and (sent[t]['feats'] and 'V' not in sent[t]['feats']) and t < len(sent):
                    t += 1
                if sent[t]['feats'] and 'S' in sent[t]['feats']:
                    pair.append(sent[t]['form'])
                if len(pair) > 1:
                    if pair[0] in model.wv.vocab and pair[1] in model.wv.vocab:
                        self.found['coordinate_NPs'].append(pair + [s, i, model.similarity(pair[0], pair[1])])
                    else:
                        self.found['coordinate_NPs'].append(pair + [s, i, float('-inf')])
                    self.found_word[(pair[0], s, i)].append('coordinate_NPs')
                    self.found_word[(pair[1], s, i+1)].append('coordinate_NPs')

    def not_in_vocabulary(self, ad, word, i, model, s):
        if word['form'].isalpha() and ad.only_alphabet_chars(word['form'], "CYRILLIC") and word[
            'form'].lower() not in model.wv.vocab:
            self.found['not in vocabulary'].append((word['form'], s, i))
            self.found_word[(word['form'], s, i)].append('not in vocabulary')

    def i_vs_we(self, i, word, s):
        if word['lemma'] == 'Я' and not self.flag_i_vs_we:
            self.flag_i_vs_we = 'i'
            self.found['i vs we'].append((word['form'], s, i))
            self.found_word[(word['form'], s, i)].append('i vs we')
        elif (word['lemma'] == 'Я' and self.flag_i_vs_we == 'we') or (
                word['lemma'] == 'МЫ' and self.flag_i_vs_we == 'i'):
            self.found['i vs we'].append((word['form'], s, i))
            self.found_word[(word['form'], s, i)].append('i vs we')
        elif word['lemma'] == 'МЫ' and not self.flag_i_vs_we:
            self.flag_i_vs_we = 'we'
            self.found['i vs we'].append((word['form'], s, i))
            self.found_word[(word['form'], s, i)].append('i vs we')

    def check_mood(self, sent, i, word, s):
        if word['form'] == 'бы' and i > 0:
            self.found['subjunctive mood'].append((sent[i - 1]['form'], word['form'], s, i))
            self.found_word[(sent[i - 1]['form'], s, i-1)].append('subjunctive mood')
            self.found_word[(word['form'], s, i)].append('subjunctive mood')
        if word['feats'] and 'Mood' in word['feats'].keys() and word['feats']['Mood'] == 'Imp':
            self.found['imperative mood'].append((word['form'], s, i))
            self.found_word[(word['form'], s, i)].append('imperative mood')


    def check_all(self, tree):
        #s - sentence number
        #i - word number
        logging.basicConfig(level=logging.INFO, filename='found.log')
        model = Word2Vec.load(MODEL)
        ad = AlphabetDetector()

        for s, sent in enumerate(tqdm(tree)):
            gen_chain = []

            for i, word in enumerate(sent):
                self.check_mood(sent, i, word, s)
                self.i_vs_we(i, word, s)
                self.not_in_vocabulary(ad, word, i, model, s)
                gen_chain = self.find_genitives(gen_chain, word, s, i)
                self.find_wrong_comparativ(sent, word, i, s)
                self.find_wrong_coordinate_NPs(sent, i, s, word, model)

        for key, value in self.found.items():
            logging.info(key)
            for mistake in value:
                logging.info(mistake)
                
        return self.found_word


In [11]:
import ufal.udpipe 

In [12]:
from conllu import parse, parse_tree

In [13]:
from ufal.udpipe import Model, Pipeline, ProcessingError

In [14]:
ud_model = Model.load('russian-syntagrus-ud-2.3-181115.udpipe')

In [15]:
sample_text = """
Рассматривайте задачу, когда у нас есть куча обучающих примеров (изображений), но, к сожалению, их забыли разметить. 
Ну и есть небольшое подмножество примеров из той же выборки, которые все же напряглись и разметили по классам. И нам необходимо научиться решать задачу классификации. Правда, в размеченной выборке на каждый класс приходится маловато примеров: конкретно от 1 до 50, так что обычный supervised learning явно не взлетит. Авторы предлагают для решения метод, который они назвали CACTUs
"""

In [16]:
pipeline = Pipeline(ud_model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

In [17]:
out = pipeline.process(sample_text)

In [18]:
tree = parse(out)

In [23]:
searcher = Searcher()

In [24]:
check = searcher.check_all(tree)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 265.90it/s]


### Найденные ошибки

In [25]:
check.items()

dict_items([(('Рассматривайте', 0, 0), ['imperative mood']), (('когда', 0, 3), ['not in vocabulary']), (('у', 0, 4), ['not in vocabulary']), (('нас', 0, 5), ['not in vocabulary']), (('есть', 0, 6), ['not in vocabulary']), (('но', 0, 14), ['not in vocabulary']), (('к', 0, 16), ['not in vocabulary']), (('их', 0, 19), ['not in vocabulary']), (('разметить', 0, 21), ['not in vocabulary']), (('Ну', 1, 0), ['not in vocabulary']), (('и', 1, 1), ['not in vocabulary']), (('есть', 1, 2), ['not in vocabulary']), (('из', 1, 6), ['not in vocabulary']), (('же', 1, 8), ['not in vocabulary']), (('все', 1, 12), ['not in vocabulary']), (('же', 1, 13), ['not in vocabulary']), (('и', 1, 15), ['not in vocabulary']), (('разметили', 1, 16), ['not in vocabulary']), (('по', 1, 17), ['not in vocabulary']), (('И', 2, 0), ['not in vocabulary']), (('в', 3, 2), ['not in vocabulary']), (('размеченной', 3, 3), ['not in vocabulary']), (('на', 3, 5), ['not in vocabulary']), (('от', 3, 13), ['not in vocabulary']), (('до'

In [26]:
def process_text(text):
    #readability_string = check_text(text)
    out = pipeline.process(text)
    tree = parse(out)
    check = searcher.check_all(tree)

    sents = []
    for s, sent in enumerate(tree):
        if sent[-1]['misc'] and sent[-1]['misc']['SpacesAfter'] and '\\n' in sent[-1]['misc']['SpacesAfter']:
            s = [i for i in sent.metadata['text'].split()]
            s.append('\n')
            sents.append(s)
        else:
            sents.append([i for i in sent.metadata['text'].split()])

    out_t = []
    for i in range(len(sents)):
        for j in range(len(sents[i])):
            if (sents[i][j], i, j) in check.keys():
                word = str(sents[i][j])
                out_t.append(style.color_scheme[check[(word, i, j)][0]].format(word))
            else:
                out_t.append(sents[i][j])

    string = ' '.join(out_t)

    return string

In [27]:
string = process_text(sample_text)

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 3338.88it/s]


In [28]:
string

'<span style="background-color: #DEB887">Рассматривайте</span> задачу, когда у нас есть куча обучающих примеров (изображений), но, к сожалению, их забыли разметить. \n <span style="background-color: #FFE4E1">Ну</span> <span style="background-color: #FFE4E1">и</span> <span style="background-color: #FFE4E1">есть</span> небольшое подмножество примеров <span style="background-color: #FFE4E1">из</span> той <span style="background-color: #FFE4E1">же</span> выборки, которые все же напряглись и разметили по классам. <span style="background-color: #FFE4E1">И</span> нам необходимо научиться решать задачу классификации. Правда, в размеченной выборке на каждый класс приходится маловато примеров: конкретно от 1 до 50, так что обычный supervised learning явно не взлетит. Авторы предлагают <span style="background-color: #FFE4E1">для</span> решения метод, который они назвали CACTUs \n'

In [29]:
HTML(string)